In [1]:
import numpy as np
import pandas as pd
import datetime

In [2]:
# dataBefore = './dataset/train_sessions.csv' #Path to Original Training Dataset "Clicks" File
dataBefore = './dataset/trpur_sessions.csv' #Path to Original Training Dataset "Clicks" File
dataTestBefore = './dataset/test_leaderboard_sessions.csv' #Path to Original Testing Dataset "Clicks" File
dataTestFinalBefore = './dataset/test_final_sessions.csv' #Path to Original Testing Dataset "Clicks" File
dataAfter = './processed/' #Path to Processed Dataset Folder
dayTime = 86400 #Validation Only one day = 86400 seconds


In [9]:

def removeShortSessions(data):
    #delete sessions of length < 1
    sessionLen = data.groupby('session_id').size() #group by sessionID and get size of each session
    data = data[np.in1d(data.session_id, sessionLen[sessionLen > 1].index)]
    return data


In [3]:
train= pd.read_csv(dataBefore)
print(train.columns)

Index(['Unnamed: 0', 'session_id', 'item_id', 'date'], dtype='object')


In [4]:
train = pd.read_csv(dataBefore, usecols=[1,2,3], dtype={0:np.int32, 1:np.int64, 2:str})
print(type(train))
print(train.columns)
print(train.head())

#Read Dataset in pandas Dataframe (Ignore Category Column)
# train = pd.read_csv(dataBefore, sep=',', dtype={0:np.int32, 1:np.int64, 2:str})
test = pd.read_csv(dataTestBefore, sep=',', dtype={0:np.int32, 1:np.int64, 2:str})
test_f = pd.read_csv(dataTestFinalBefore, sep=',', dtype={0:np.int32, 1:np.int64, 2:str})
# train.columns = ['SessionID','ItemID', 'Time', ] #Headers of dataframe
# test.columns = ['SessionID', 'ItemID', 'Time', ] #Headers of dataframe
train['date'] = pd.to_datetime(train['date'], format='%Y-%m-%d %H:%M:%S')
test['date'] = pd.to_datetime(test['date'], format='%Y-%m-%d %H:%M:%S')
test_f['date'] = pd.to_datetime(test_f['date'], format='%Y-%m-%d %H:%M:%S')

<class 'pandas.core.frame.DataFrame'>
Index(['session_id', 'item_id', 'date'], dtype='object')
   session_id item_id                     date
0           3    9655  2020-12-18 21:19:48.093
1           3    9655  2020-12-18 21:25:00.373
2           3   15085  2020-12-18 21:26:47.986
3          13   15654  2020-03-13 19:35:27.136
4          13   18626  2020-03-13 19:36:15.507


In [5]:
## convert to timestamp
import time
train['date'] = train['date'].apply(lambda date: time.mktime(date.timetuple()))
test['date'] = test['date'].apply(lambda date: time.mktime(date.timetuple()))
test_f['date'] = test_f['date'].apply(lambda date: time.mktime(date.timetuple()))
print(test_f.head())

   session_id  item_id          date
0          61    27088  1.622503e+09
1          96    11693  1.624092e+09
2          96    18298  1.624093e+09
3          96     4738  1.624093e+09
4          96      495  1.624093e+09


In [ ]:
# #sort by date
# train = train.groupby(['session_id']).apply(lambda x: x.sort_values(['date'], ascending=True))
# print(train.head())

In [27]:
#remove sessions of less than 2 interactions
train = removeShortSessions(train)

# #delete records of items which appeared less than 5 times
# itemLen = train.groupby('item_id').size() #groupby item_id and get size of each item
# train = train[np.in1d(train.item_id, itemLen[itemLen > 4].index)]

#remove sessions of less than 2 interactions again
train = removeShortSessions(train)

In [28]:
# #Delete records in testing split where items are not in training split TODO
# test = test[np.in1d(test.item_id, train.item_id)]

#Delete Sessions in testing split which are less than 2
test = removeShortSessions(test)


In [6]:
#Convert To CSV
print('Full Training Set has', len(train), 'Events, ', train.session_id.nunique(), 'Sessions, and', train.item_id.nunique(), 'Items\n\n')
#train.to_csv(dataAfter + 'recSys15TrainFull.txt', sep='\t', index=False)
print('Testing Set has', len(test), 'Events, ', test.session_id.nunique(), 'Sessions, and', test.item_id.nunique(), 'Items\n\n')
test.to_csv(dataAfter + 'test.txt', sep=',', index=False)

print('Testing Set has', len(test_f), 'Events, ', test_f.session_id.nunique(), 'Sessions, and', test_f.item_id.nunique(), 'Items\n\n')
test_f.to_csv(dataAfter + 'test_f.txt', sep=',', index=False)


Full Training Set has 5743820 Events,  1000000 Sessions, and 23618 Items


Testing Set has 229354 Events,  50000 Sessions, and 5647 Items


Testing Set has 226138 Events,  50000 Sessions, and 5648 Items




In [10]:
#Separate Training set into Train and Validation Splits
# timeMax = train.date.max()
# sessionMaxTime = train.groupby('session_id').date.max()
# sessionTrain = sessionMaxTime[sessionMaxTime < (timeMax - dayTime)].index #training split is all sessions that ended before the last 2nd day
# sessionValid = sessionMaxTime[sessionMaxTime >= (timeMax - dayTime)].index #validation split is all sessions that ended during the last 2nd day
# trainTR = train[np.in1d(train.SessionID, sessionTrain)]
# trainVD = train[np.in1d(train.SessionID, sessionValid)]
# #Delete records in validation split where items are not in training split
# trainVD = trainVD[np.in1d(trainVD.ItemID, trainTR.ItemID)]
# #Delete Sessions in testing split which are less than 2

splitlen = int(len(train) * 0.9)
trainTR = train.iloc[:splitlen,:] #first 0.9
trainVD = train.iloc[splitlen:,:] #last 0.1

trainVD = removeShortSessions(trainVD)
#Convert To CSV
print('Training Set has', len(trainTR), 'Events, ', trainTR.session_id.nunique(), 'Sessions, and', trainTR.item_id.nunique(), 'Items\n\n')
trainTR.to_csv(dataAfter + 'train.txt', sep=',', index=False)
print('Validation Set has', len(trainVD), 'Events, ', trainVD.session_id.nunique(), 'Sessions, and', trainVD.item_id.nunique(), 'Items\n\n')
trainVD.to_csv(dataAfter + 'valid.txt', sep=',', index=False)

Training Set has 4595056 Events,  799608 Sessions, and 23119 Items


Validation Set has 1148764 Events,  200393 Sessions, and 20412 Items




# 1. nn.Embedding for item categorica features

In [45]:
import torch
import torch.nn as nn

n, d, m = 3, 5, 7
embedding = nn.Embedding(n, d, max_norm=True)
print(embedding.weight)


Parameter containing:
tensor([[ 0.5856,  0.9176, -0.7671, -0.3828,  0.4716],
        [-1.2720,  0.0956, -2.0452,  0.1737, -0.2470],
        [-0.8038,  1.9868, -0.6960,  1.3874,  2.3874]], requires_grad=True)


In [52]:
W = torch.randn((m, d), requires_grad=True)
print(W.t())
idx = torch.tensor([1, 2])
a = embedding.weight.clone() @ W.t()  # weight must be cloned for this to be differentiable
print(a)
b = embedding(idx) @ W.t()  # modifies weight in-place
print(b)
out = (a.unsqueeze(0) + b.unsqueeze(1))
loss = out.sigmoid().prod()
print(loss)
loss.backward()

tensor([[-0.4476,  1.2224,  0.5746,  0.7182,  0.9629,  0.0524, -0.7697],
        [-1.7727,  0.6874, -1.2663,  0.4730, -0.7384,  0.8012, -0.7183],
        [ 0.3871, -0.0684,  2.4013,  0.0066, -0.8426,  1.2806,  0.8540],
        [-0.9335, -0.9808,  0.3194,  0.1850,  0.8160, -0.3397,  1.5963],
        [-0.9586, -0.3076,  0.1421,  0.3562,  0.6051, -0.1612, -0.1200]],
       grad_fn=<TBackward0>)
tensor([[-2.2805,  1.6296, -2.7227,  0.9467,  0.5055, -0.1623, -2.4327],
        [-0.1306, -0.5943, -2.3640, -0.3860,  0.1730, -1.0819, -0.2179],
        [-1.9683, -0.4670, -1.0849,  0.4111,  0.2587, -0.0554,  0.1474]],
       grad_fn=<MmBackward0>)
tensor([[-0.1306, -0.5943, -2.3640, -0.3860,  0.1730, -1.0819, -0.2179],
        [-1.9683, -0.4670, -1.0849,  0.4111,  0.2587, -0.0554,  0.1474]],
       grad_fn=<MmBackward0>)
tensor(5.0869e-30, grad_fn=<ProdBackward0>)


In [53]:
weight = torch.FloatTensor([[1, 0, 0], [0, 1, 1]])
embedding = nn.Embedding.from_pretrained(weight)
input = torch.LongTensor([1])
embedding(input)

tensor([[0., 1., 1.]])